In [1]:
from pathlib import Path 

import IMP
import IMP.atom 

In [6]:
pdb_file = Path(Path.home(), "xray/md_xray/2h2z_100k/output_0/pdbs/md_0.pdb")

In [7]:
m = IMP.Model() 
h = IMP.atom.read_pdb(str(pdb_file), m)

In [10]:
m_test = h.get_model()
len(m_test.get_particle_indexes())

2680